In [4]:
import requests

response = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-1000-titres-les-plus-reserves-dans-les-bibliotheques-de-pret/records?limit=100")

# Afficher le type de contenu et le texte brut de la réponse
print("Type de contenu :", response.headers.get("Content-Type"))

Type de contenu : application/json; charset=utf-8


On met comme limite 100 pcq si on ne le fait pas, il y a un message d'erreur qui dit que la limite doit être entre -1 et 100

In [5]:
wb = response.json()  # Utilisation de .json() sur l'objet réponse
print("Structure du JSON :")
print(wb)

Structure du JSON :
{'total_count': 1000, 'results': [{'rang': 12, 'type_de_document': 'Livre adulte', 'reservations': 522.0, 'titre': "Réinventer l'amour : comment le patriarcat sabote les relations hétérosexuelles", 'auteur': 'Chollet,  Mona'}, {'rang': 13, 'type_de_document': 'Bande dessinée adulte', 'reservations': 521.0, 'titre': 'Le jeune acteur', 'auteur': 'Sattouf,  Riad'}, {'rang': 14, 'type_de_document': 'Bande dessinée adulte', 'reservations': 510.0, 'titre': 'Une jeunesse au Moyen-Orient, 1992-1994', 'auteur': 'Sattouf,  Riad'}, {'rang': 37, 'type_de_document': 'DVD tous publics', 'reservations': 337.0, 'titre': 'Dune', 'auteur': None}, {'rang': 58, 'type_de_document': 'DVD tous publics', 'reservations': 266.0, 'titre': 'Onoda. 10 000 nuits dans la jungle', 'auteur': None}, {'rang': 73, 'type_de_document': 'Nouveauté', 'reservations': 241.0, 'titre': 'Chien 51 : roman', 'auteur': 'Gaudé,  Laurent'}, {'rang': 87, 'type_de_document': 'Livre adulte', 'reservations': 225.0, 'ti

Question: pourquoi est-ce qu'il y a pas tous les rangs ???

In [9]:
import pandas as pd
df= pd.json_normalize(wb['results'])
df.head(5)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
3,37,DVD tous publics,337.0,Dune,None
4,58,DVD tous publics,266.0,Onoda. 10 000 nuits dans la jungle,None


In [11]:
print(df["type_de_document"].unique())

['Livre adulte' 'Bande dessinée adulte' 'DVD tous publics' 'Nouveauté'
 'Bande dessinée jeunesse' 'DVD nouveautés tous publics' None
 'Nouveauté jeunesse' 'Bande dessinée ado' 'Livre ado' 'Livre jeunesse']


In [12]:
doc = ["Livre adulte", "Bande dessinée adulte", "Nouveauté", "Bande dessinée jeunesse", "Nouveauté jeunesse",
 "Bande dessinée ado", "Livre ado", "Livre jeunesse"]
df_filtered= df[df["type_de_document"].isin(doc)]

In [13]:
df_filtered.head(3)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"


In [14]:
nouveau = ["Nouveauté", "Nouveauté jeunesse"]
df_nouveau =  df[df["type_de_document"].isin(nouveau)]
print(df_nouveau)

    rang    type_de_document  reservations  \
5     73           Nouveauté         241.0   
19   394  Nouveauté jeunesse          95.0   
20   399           Nouveauté          95.0   
22   425  Nouveauté jeunesse          91.0   
34   672           Nouveauté          71.0   
40   720  Nouveauté jeunesse          68.0   
49   910           Nouveauté          60.0   
51   945           Nouveauté          58.0   
52   949           Nouveauté          58.0   
66   217           Nouveauté         132.0   
69   233           Nouveauté         127.0   
71   275           Nouveauté         114.0   
92   726           Nouveauté          68.0   
96   762           Nouveauté          66.0   

                                                titre                 auteur  
5                                    Chien 51 : roman        Gaudé,  Laurent  
19      My hero academia. 8. Momo Yaoyorozu : l'envol      Horikoshi,  Kohei  
20                                 Le poids des héros           Sala,  D

Le but maintenant va être de scrapper wikipedia pour compléter la base et les catégories "nouveaux" qui ne renseignent pas sur le type d'ouvrage

In [1]:
import requests

url_ligue_1 = (
    "https://fr.wikipedia.org/wiki/Laurent_Gaud%C3%A9"
)

request_text = requests.get(url_ligue_1).content

In [19]:
print(request_text)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Laurent Gaud\xc3\xa9 \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limit

In [8]:
import bs4
page = bs4.BeautifulSoup(request_text, "html.parser")

In [10]:
print(page.find("table"))

<table><caption style="color:#FFFFFF;text-align:center;background-color:#4C6099">Biographie</caption><tbody><tr class=""><th scope="row">Naissance</th><td class=""><div>
<time class="nowrap date-lien bday" data-sort-value="1972-07-06" datetime="1972-07-06"><a href="/wiki/6_juillet" title="6 juillet">6</a> <a href="/wiki/Juillet_1972" title="Juillet 1972">juillet</a> <a href="/wiki/1972_en_litt%C3%A9rature" title="1972 en littérature">1972</a></time> <span class="noprint">(52 ans)</span><br/><a href="/wiki/Paris" title="Paris">Paris</a></div></td></tr><tr class=""><th scope="row">Nationalité</th><td class=""><div>
<span class="wd_p27"><a href="/wiki/France" title="France">française</a><span class="noprint wikidata-linkback skin-invert"><span class="mw-valign-baseline noviewer" typeof="mw:File"><a href="https://www.wikidata.org/wiki/Q1245984?uselang=fr#P27" title="Voir et modifier les données sur Wikidata"><img alt="Voir et modifier les données sur Wikidata" class="mw-file-element" data-

In [ ]:
import requests
from bs4 import BeautifulSoup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_babelio(book_title):
    # URL de recherche sur Babelio
    search_url = f'https://www.babelio.com/recherche?search={book_title}'

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        search_page = BeautifulSoup(response.text, 'html.parser')
        
        # Trouver le premier lien vers un livre dans les résultats
        first_result = search_page.find('a', class_='cover')
        
        if first_result:
            # Extraire l'URL de la page du premier résultat
            first_result_url = first_result.get('href')
            print(f'URL du premier résultat: {first_result_url}')
            
            # Accéder à la page du premier résultat
            book_page_url = f'https://www.babelio.com{first_result_url}'
            response_book_page = requests.get(book_page_url)
            
            if response_book_page.status_code == 200:
                book_soup = BeautifulSoup(response_book_page.text, 'html.parser')
                
                # Scraper des informations sur le livre depuis la page
                book_title = book_soup.find('h1', class_='bookTitle').get_text(strip=True) if book_soup.find('h1', class_='bookTitle') else 'Titre non trouvé'
                author_name = book_soup.find('span', itemprop='author').get_text(strip=True) if book_soup.find('span', itemprop='author') else 'Auteur non trouvé'
                description = book_soup.find('div', class_='biographie').get_text(strip=True) if book_soup.find('div', class_='biographie') else 'Description non trouvée'
                
                # Afficher les informations récupérées
                print(f'Titre: {book_title}')
                print(f'Auteur: {author_name}')
                print(f'Description: {description[:200]}...')  # Afficher les 200 premiers caractères de la description
            else:
                print(f"Erreur lors de l'accès à la page du livre: {response_book_page.status_code}")
        else:
            print('Premier résultat non trouvé.')
    else:
        print(f"Erreur lors de la recherche: {response.status_code}")

# Recherche d'un livre par son titre
search_babelio("Le Petit Prince")


In [8]:
import requests
from bs4 import BeautifulSoup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_livraddict(book_title):
    # URL de recherche sur Babelio
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.livraddict.com/search.php?t={book_title}]'

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response, "html.parser")
    first_result = page.select_one('.listing_recherche li .item_photo a')
    url = first_result['href']
    full_url = f"https://www.livraddict.com{url}"
    return full_url

In [25]:
def obtain_type(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).content
    page = BeautifulSoup(response, "html.parser")
    c_format= page.find("div", class_= "book_classif book_format mr1")
    c_lectorat= page.find("div", class_="book_classif book_lectorat mr1")
    if c_format:
        a_format = c_format.find("a")
        format_livre = a_format.text
    else:
        format_livre = "Non renseigné"
    if c_lectorat:
        a_lectorat = c_lectorat.find("a")
        lectorat = a_lectorat.text
    else: 
        lectorat = "Non renseigné"
    info = [format_livre, lectorat]
    return info

In [ ]:
def type_livre:
    if info[0] in []

TEST AVEC DES EXEMPLES SIMPLES

In [62]:
search_livraddict("Percy+Jackson")

'https://www.livraddict.com/biblio/livre/percy-jackson-comics-tome-1-le-voleur-de-foudre.html'

In [26]:
obtain_type("https://www.livraddict.com/biblio/livre/percy-jackson-comics-tome-1-le-voleur-de-foudre.html")

[' Comics', ' Adolescence']